# Yaw Robustness Analysis

This notebook generates:
1. Success Rate vs Yaw Noise bar plot
2. Break Rate vs Yaw Noise bar plot

In [ ]:
# ============================================================
# BLOCK 1: IMPORTS & LOCAL CONFIGURATION
# ============================================================

import wandb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import shared utilities
from analysis_utils import (
    # Constants
    ENTITY, PROJECT, COLORS,
    TAG_EVAL_PERFORMANCE, TAG_EVAL_YAW,
    METRIC_SUCCESS, METRIC_BREAKS, METRIC_TOTAL,
    # Data functions
    get_best_checkpoint_per_run,
    download_eval_data,
    # Plotting functions
    plot_rate_figure,
    print_data_summary,
)

# ============================================================
# LOCAL CONFIGURATION (specific to this analysis)
# ============================================================

# Method Tags
METHOD_TAGS = {
    "Pose(3deg)": "pose_hex_state-std:2026-01-26_18:36",
    "MATCH(3deg)": "MATCH_hex_state-std:2026-01-26_17:33",
}

# Yaw Level Mapping: display label -> metric range string
# NOTE: The 0deg range has a leading space in wandb: " 0deg-15deg"
YAW_LEVELS = {
    "15deg": " 0deg-15deg",
    "30deg": "15deg-30deg",
    "45deg": "30deg-45deg",
    "60deg": "45deg-60deg",
}

# Policy Selection
TOP_N_POLICIES = None
MAX_CHECKPOINT = None  # Set to int to limit checkpoint search (e.g., 2000000 for first 2M steps)

# Plot Configuration
SUCCESS_Y_LIM = (0, 100)
SUCCESS_Y_TICKS = [0, 20, 40, 60, 80, 100]
BREAK_Y_LIM = (0, 15)
BREAK_Y_TICKS = list(range(15))

# Error type: "ci" for 95% confidence interval, "binary_se" for binary standard error
ERROR_TYPE = "ci"

In [ ]:
# ============================================================
# BLOCK 2: DETERMINE BEST POLICY
# ============================================================

api = wandb.Api()
best_checkpoints_by_method = {}

for method_name, method_tag in METHOD_TAGS.items():
    print(f"\n{method_name} ({method_tag}):")
    best_checkpoints_by_method[method_name] = get_best_checkpoint_per_run(
        api, method_tag, max_checkpoint=MAX_CHECKPOINT
    )

In [ ]:
# ============================================================
# BLOCK 3: DOWNLOAD DATA
# ============================================================

yaw_data = {}
for method_name, method_tag in METHOD_TAGS.items():
    print(f"\nDownloading data for {method_name}...")
    yaw_data[method_name] = download_eval_data(
        api=api,
        method_tag=method_tag,
        best_checkpoints=best_checkpoints_by_method[method_name],
        level_mapping=YAW_LEVELS,
        prefix_template="Yaw_Eval({level})_Core",
        level_col_name="yaw_level",
        eval_tag=TAG_EVAL_YAW,
    )

# Print summary
print_data_summary(
    data=yaw_data,
    level_labels=list(YAW_LEVELS.keys()),
    level_col="yaw_level",
    metric="success",
    title="YAW DATA SUMMARY (Success Rate)",
)

In [ ]:
# ============================================================
# BLOCK 4: SUCCESS RATE VS YAW NOISE
# ============================================================

fig, ax = plot_rate_figure(
    data=yaw_data,
    method_names=list(METHOD_TAGS.keys()),
    level_labels=list(YAW_LEVELS.keys()),
    level_col="yaw_level",
    metric="success",
    title="Success Rate vs Yaw Noise",
    x_label="Yaw Noise",
    y_label="Success Rate (%)",
    y_lim=SUCCESS_Y_LIM,
    y_ticks=SUCCESS_Y_TICKS,
    error_type=ERROR_TYPE,
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=best_checkpoints_by_method,
)
plt.show()

In [ ]:
# ============================================================
# BLOCK 5: BREAK RATE VS YAW NOISE
# ============================================================

fig, ax = plot_rate_figure(
    data=yaw_data,
    method_names=list(METHOD_TAGS.keys()),
    level_labels=list(YAW_LEVELS.keys()),
    level_col="yaw_level",
    metric="breaks",
    title="Break Rate vs Yaw Noise",
    x_label="Yaw Noise",
    y_label="Break Rate (%)",
    y_lim=BREAK_Y_LIM,
    y_ticks=BREAK_Y_TICKS,
    error_type=ERROR_TYPE,
    filter_top_n=TOP_N_POLICIES,
    best_checkpoints=best_checkpoints_by_method,
)
plt.show()